<a href="https://colab.research.google.com/github/vignesh99/ST-CS6300/blob/master/HMMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [147]:
                                    #Get data files onto root
%cd
from google.colab import drive
drive.mount('/content/gdrive')
!cp -avr /content/gdrive/My\ Drive/HMM_IsoData/1 /root
!cp -avr /content/gdrive/My\ Drive/HMM_IsoData/2 /root
!cp -avr /content/gdrive/My\ Drive/HMM_IsoData/3 /root
!cp -avr /content/gdrive/My\ Drive/HMM_IsoData/6 /root
!cp -avr /content/gdrive/My\ Drive/HMM_IsoData/7 /root
!cp -avr /content/gdrive/My\ Drive/HMM_CtsData/dev /root
!cp -avr /content/gdrive/My\ Drive/HMM_CtsData/test /root
!cp -avr /content/gdrive/My\ Drive/hmm-1.04 /root
!ls

/root
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
removed '/root/1'
'/content/gdrive/My Drive/HMM_IsoData/1' -> '/root/1'
removed '/root/2'
'/content/gdrive/My Drive/HMM_IsoData/2' -> '/root/2'
removed '/root/3'
'/content/gdrive/My Drive/HMM_IsoData/3' -> '/root/3'
removed '/root/6'
'/content/gdrive/My Drive/HMM_IsoData/6' -> '/root/6'
removed '/root/7'
'/content/gdrive/My Drive/HMM_IsoData/7' -> '/root/7'
removed '/root/dev/team15'
'/content/gdrive/My Drive/HMM_CtsData/dev/team15' -> '/root/dev/team15'
removed '/root/test/team15'
'/content/gdrive/My Drive/HMM_CtsData/test/team15' -> '/root/test/team15'
'/content/gdrive/My Drive/hmm-1.04/test.hmm' -> '/root/hmm-1.04/test.hmm'
'/content/gdrive/My Drive/hmm-1.04/README.hmm' -> '/root/hmm-1.04/README.hmm'
'/content/gdrive/My Drive/hmm-1.04/makefile.old' -> '/root/hmm-1.04/makefile.old'
'/content/gdrive/My Drive/hmm-1.04/hmm.h' -> '/root/hmm-1.04/hmm.h'


In [276]:
                                    #Test file
!cp -avr /root/1/dev/ra_1.mfcc /root
filex = "ra_1.mfcc"

'/root/1/dev/ra_1.mfcc' -> '/root/ra_1.mfcc'


In [277]:
                                    #Import required libraries
from pylab import *
import os                           #Going through folders
from scipy.cluster.vq import kmeans2
from google.colab import files

In [278]:
def trainKmeans(fileMfcc,Nclust=12) :
                                    #Convert features from file into array
    features = np.genfromtxt(fileMfcc,delimiter='\n',dtype=str)
    Nc,Nf = features[0].split(" ")
    Nc = int(Nc)
    Nf = int(Nf)
    features = np.genfromtxt(features[1:],delimiter=' ',dtype=str)
    features = features.astype(float)
    
                                    #Apply K-means on the vectors
    if len(features) == Nf and len(features[0]) == Nc :
        centroid,label = kmeans2(features,Nclust)

    return label,features


In [279]:
label,ft = trainKmeans(filex)
label

/usr/local/lib/python3.6/dist-packages/scipy/cluster/vq.py:579: UserWarning: One of the clusters is empty. Re-run kmeans with a different initialization.
  warnings.warn("One of the clusters is empty. "


array([11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11,  9,  9,  9,  9, 11,  9,  9, 11, 11, 11, 11, 11, 11,
       11, 11,  4,  4,  3,  3,  3,  3,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        7,  6,  6,  6,  6,  6,  6,  6,  6,  6,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  5,  5,  5,  5,  5,  5,  5,  1,  1,  1,  1,  1,  1,  1,
       10, 10, 10, 10, 10, 10,  1,  1,  1,  4,  4,  4,  4,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2, 11,  2,  2,  2,  2,  4,  2, 11,
       11, 11], dtype=int32)

In [280]:
def seqDwnld(seq,fileWord) :
    with open(fileWord,"w") as f:
        f.write("\n".join(" ".join(map(str, line)) for line in seq))

    files.download(fileWord)

    return None

In [281]:
                                    #Give the file with sequences for training data
def trainSeq(word,fileWord="check.txt") :
    seq = []
    count = 0
                                    #Go through required folders
    for subdir, dirs, files in os.walk(word):
        dirname = subdir.split(os.path.sep)[-1]
        print('Directory:', dirname)
        if dirname == "train":
            for f in files:
                if f.endswith('.mfcc'):
                                    #Copy file (mfcc) to root and perform K-means
                    !cp -avr /root/$word/train/$f /root
                    label,temp = trainKmeans(f)
                    seq.append(label)
                                    #Concatenate training feature vectors
                    if count == 0 :
                        features = temp
                    else :
                        features = np.concatenate((features,temp))
                    !rm $f
                    count+=1
    seq = array(seq)

    seqDwnld(seq,fileWord)

    return features


In [282]:
!ls

1  2  3  6  7  dev  hmm-1.04  ra_1.mfcc  test


In [288]:
def trainWords(Nclust=12) :
    oneF = trainSeq("1","one.seq")
    #rmfile = "one.seq"
    #!rm $rmfile
                                    #Obtain cluster centres of each word
    oneC,label = kmeans2(oneF,Nclust)
    #trainSeq("2","two.seq") 
    #trainSeq("3","three.seq") 
    #trainSeq("6","six.seq") 
    #trainSeq("7","seven.seq") 

    return oneC

In [293]:
oneC = trainWords()
print(shape(oneC))

Directory: 1
Directory: train
'/root/1/train/ac_1.mfcc' -> '/root/ac_1.mfcc'


/usr/local/lib/python3.6/dist-packages/scipy/cluster/vq.py:579: UserWarning: One of the clusters is empty. Re-run kmeans with a different initialization.
  warnings.warn("One of the clusters is empty. "


'/root/1/train/ag_1.mfcc' -> '/root/ag_1.mfcc'
'/root/1/train/ai_1.mfcc' -> '/root/ai_1.mfcc'
'/root/1/train/an_1.mfcc' -> '/root/an_1.mfcc'
'/root/1/train/bh_1.mfcc' -> '/root/bh_1.mfcc'
'/root/1/train/bi_1.mfcc' -> '/root/bi_1.mfcc'
'/root/1/train/br_1.mfcc' -> '/root/br_1.mfcc'
'/root/1/train/ca_1.mfcc' -> '/root/ca_1.mfcc'
'/root/1/train/cg_1.mfcc' -> '/root/cg_1.mfcc'
'/root/1/train/cl_1.mfcc' -> '/root/cl_1.mfcc'
'/root/1/train/cm_1.mfcc' -> '/root/cm_1.mfcc'
'/root/1/train/dc_1.mfcc' -> '/root/dc_1.mfcc'
'/root/1/train/dg_1.mfcc' -> '/root/dg_1.mfcc'
'/root/1/train/ea_1.mfcc' -> '/root/ea_1.mfcc'
'/root/1/train/ec_1.mfcc' -> '/root/ec_1.mfcc'
'/root/1/train/ee_1.mfcc' -> '/root/ee_1.mfcc'
'/root/1/train/eg_1.mfcc' -> '/root/eg_1.mfcc'
'/root/1/train/ei_1.mfcc' -> '/root/ei_1.mfcc'
'/root/1/train/ek_1.mfcc' -> '/root/ek_1.mfcc'
'/root/1/train/es_1.mfcc' -> '/root/es_1.mfcc'
'/root/1/train/hg_1.mfcc' -> '/root/hg_1.mfcc'
'/root/1/train/hp_1.mfcc' -> '/root/hp_1.mfcc'
'/root/1/trai

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

(12, 38)


In [294]:
!ls

1  2  3  6  7  dev  hmm-1.04  one.seq  ra_1.mfcc  test


In [302]:
def testKmeans(fileMfcc,cenCl,Nclust=12,iters=1) :
                                    #Convert features from file into array
    features = np.genfromtxt(fileMfcc,delimiter='\n',dtype=str)
    Nc,Nf = features[0].split(" ")
    Nc = int(Nc)
    Nf = int(Nf)
    features = np.genfromtxt(features[1:],delimiter=' ',dtype=str)
    features = features.astype(float)
    
                                    #Apply K-means on the vectors
    if len(features) == Nf and len(features[0]) == Nc :
        centroid,label = kmeans2(features,k=cenCl,iter=iters,minit="matrix")

    return label


In [312]:
                                    #Give the file with sequences for testing data
def testSeq(cenCl,fileWord="check.txt",folders=None) :
    if folders is None :
        folders = array(["1","2","3","6","7"])
    seq = []
    count = np.zeros(5)
    for word in folders :    
                                        #Go through required folders
        for subdir, dirs, files in os.walk(word):
            dirname = subdir.split(os.path.sep)[-1]
            print('Directory:', dirname)
            if dirname == "dev":
                for f in files:
                    if f.endswith(".mfcc"):
                                        #Copy file (mfcc) to root and perform K-means
                        !cp -avr /root/$word/dev/$f /root
                        label = testKmeans(f,cenCl)
                        seq.append(label)
                        !rm $f
                        count[np.where(word == folders)[0][0]] += 1

    seq = array(seq)
    
    seqDwnld(seq,fileWord)

    return None


In [ ]:
testSeq(oneC,"one.test")

In [ ]:
def testProb(fileProb) :
    prob = np.genfromtxt(features[1:],delimiter=' ',dtype=str)
    prob = prob.astype(float)

    return prob

In [315]:
!ls

1  2  3  6  7  dev  hmm-1.04  nh_2.mfcc  one.seq  one.test  test


In [317]:
def accModel(count) :

    error = np.zeros(5)
    prob1 = testProb("fp1.txt")
    allprob = np.zeros((len(prob1),5))
    allprob[:,0] = prob1

    allprob[:,1] = testProb("fp2.txt")
    allprob[:,2] = testProb("fp3.txt")
    allprob[:,3] = testProb("fp6.txt")
    allprob[:,4] = testProb("fp7.txt")

    estimate = np.argmin(allprob,axis=1)
    for i in range(0,5) :
        error[i] = len(np.where(estimate[:count[i]]!=i))

    acc = 1-(sum(error)/sum(count))
    
    return acc